In [51]:
import requests
from bs4 import BeautifulSoup
from models import bart as model

def get_html(url, debug=False):
    response = requests.get(url)
    html = response.text
    
    if debug:
        with open('temp.html', 'w') as file:
            file.write(html)
            
    return html

def extract_text_from_html(html, debug=False):
    soup = BeautifulSoup(html, 'html.parser')
    for a in soup.find_all('a'):
       a.decompose()
    text = soup.get_text(separator='\n')
    
    if debug:
        with open('temp.txt', 'w') as file:
            file.write(text)
    
    return text

def get_summary_from_website(url, debug=False):
    html = get_html(url, debug)
    txt = extract_text_from_html(html, debug)
    summary = model.summarize(txt)
    return summary


In [ ]:
url = 'https://brimfinancial.com/'
get_summary_from_website(url, True)